In [12]:
import json
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
# from sklearn.linear_model import LinearRegression
mpl.rcParams['figure.dpi']= 200
%config InlineBackend.figure_format = 'svg'
# plt.rcParams['font.size'] = 15
# mpl.rcParams['axes.linewidth'] = 2

RYTOEV = 13.605698 # Rydberg to eV
ryd2mev = RYTOEV*1000

In [13]:
def read_ibz2bz(fname):
    m = []
    with open(fname,"r") as f:
        f.readline()
        line = f.readline()
        while line != "":
            m.append(int(line.split(',')[1]))
            line = f.readline()
    return m

def load_equivalence(prefix,nqibz):
    mappings = []
    for iq in range(nqibz):
        fname = prefix+"ibz2bz_%05d.csv"%(iq+1)
        mapping = read_ibz2bz(fname)
        mappings.append(mapping)
    return mappings

# EPCE computed with WELPH with Ponce lattice constant

In [14]:
nqibz = 16
nkibz = 16
nk = 6**3
nq = 6**3
fm = pd.read_csv("./60Ry-Ponce-latt/c.welph.save.pdep200/EPCE_DFT_FM.tab",
                   skiprows=1,names=["ikr","ibn","iq","imode","en"],sep="\\s+")
dw = pd.read_csv("./60Ry-Ponce-latt/c.welph.save.pdep200/EPCE_DFT_DW.tab",
                   skiprows=1,names=["ikr","ibn","iq","imode","en"],sep="\\s+")
mappings = load_equivalence("./60Ry-Ponce-latt/",nqibz)

In [15]:
for ikibz in [1,4]:
    for iqibz in [1,4]:
        filter1 = fm["ikr"] == ikibz
        filter2 = fm["iq"].isin(mappings[iqibz-1])
        fmpart = fm[filter1 & filter2]
        filter3 = dw["ikr"] == ikibz
        filter4 = dw["iq"].isin(mappings[iqibz-1])
        dwpart = dw[filter3 & filter4]
        q_degeneracy = len(mappings[iqibz-1])
        print("ikibz = %4d, iqibz = %4d"%(ikibz,iqibz))
#         print(fmpart[fmpart["ibn"]==1])
#         print(fmpart.groupby("ibn")["en"].sum()/q_degeneracy)
#         print(dwpart.groupby("ibn")["en"].sum()/q_degeneracy)    
        print(fmpart.groupby("ibn")["en"].sum()/q_degeneracy+dwpart.groupby("ibn")["en"].sum()/q_degeneracy)

ikibz =    1, iqibz =    1
ibn
1   -12.838822
2    24.862303
3    24.862416
4    24.862490
5   -14.879415
6   -14.879730
7   -14.877916
8   -30.864836
Name: en, dtype: float64
ikibz =    1, iqibz =    4
ibn
1    -55.147118
2    186.713634
3    186.712410
4    186.712257
5   -274.858591
6   -274.859511
7   -274.853433
8   -294.701784
Name: en, dtype: float64
ikibz =    4, iqibz =    1
ibn
1    -21.539568
2    -16.912003
3     10.023133
4     10.023246
5   -182.879295
6   -182.880274
7    365.961169
8    -26.443691
Name: en, dtype: float64
ikibz =    4, iqibz =    4
ibn
1    -91.960216
2   -224.149858
3    -27.134306
4    -27.134582
5    164.072007
6    164.073465
7   -102.771014
8   -202.567719
Name: en, dtype: float64


In [16]:
vbm_fm = fm[np.logical_and(fm["ikr"]==1,fm["ibn"]==4)]["en"].sum()/nq
vbm_dw = dw[np.logical_and(dw["ikr"]==1,dw["ibn"]==4)]["en"].sum()/nq
cbm_fm = fm[np.logical_and(fm["ikr"]==1,fm["ibn"]==5)]["en"].sum()/nq
cbm_dw = dw[np.logical_and(dw["ikr"]==1,dw["ibn"]==5)]["en"].sum()/nq

print("%10s%10s%10s%10s"%("","FM","DW","FM+DW"))
print("%10s%10.0f%10.0f%10.0f"%("VBM",vbm_fm,vbm_dw,vbm_fm+vbm_dw))
print("%10s%10.0f%10.0f%10.0f"%("CBM",cbm_fm,cbm_dw,cbm_fm+cbm_dw))
print("Direct gap ZPR = %10.0f"%((cbm_fm+cbm_dw)-(vbm_fm+vbm_dw)))

                  FM        DW     FM+DW
       VBM      -993      1120       127
       CBM     -1173      1008      -165
Direct gap ZPR =       -292


# EPCE computed with WELPH with my relaxed lattice constant

In [17]:
nqibz = 16
nkibz = 16
nk = 6**3
nq = 6**3
fm = pd.read_csv("./60Ry-myrelax/c.welph.save.pdep200/EPCE_DFT_FM.tab",
                   skiprows=1,names=["ikr","ibn","iq","imode","en"],sep="\\s+")
dw = pd.read_csv("./60Ry-myrelax/c.welph.save.pdep200/EPCE_DFT_DW.tab",
                   skiprows=1,names=["ikr","ibn","iq","imode","en"],sep="\\s+")
mappings = load_equivalence("./60Ry-myrelax/",nqibz)

In [18]:
for ikibz in [1,4]:
    for iqibz in [1,4]:
        filter1 = fm["ikr"] == ikibz
        filter2 = fm["iq"].isin(mappings[iqibz-1])
        fmpart = fm[filter1 & filter2]
        filter3 = dw["ikr"] == ikibz
        filter4 = dw["iq"].isin(mappings[iqibz-1])
        dwpart = dw[filter3 & filter4]
        q_degeneracy = len(mappings[iqibz-1])
        print("ikibz = %4d, iqibz = %4d"%(ikibz,iqibz))
#         print(fmpart[fmpart["ibn"]==1])
#         print(fmpart.groupby("ibn")["en"].sum()/q_degeneracy)
#         print(dwpart.groupby("ibn")["en"].sum()/q_degeneracy)    
        print(fmpart.groupby("ibn")["en"].sum()/q_degeneracy+dwpart.groupby("ibn")["en"].sum()/q_degeneracy)

ikibz =    1, iqibz =    1
ibn
1   -12.547744
2    25.134643
3    25.134749
4    25.134686
5   -14.872584
6   -14.870677
7   -14.871953
8   -31.905384
Name: en, dtype: float64
ikibz =    1, iqibz =    4
ibn
1    -54.469970
2    186.168082
3    186.167360
4    186.167420
5   -273.157455
6   -273.154400
7   -273.160651
8   -311.896046
Name: en, dtype: float64
ikibz =    4, iqibz =    1
ibn
1    -21.183050
2    -16.724013
3     10.137890
4     10.138035
5   -162.658589
6   -162.659740
7    324.166263
8    -26.746727
Name: en, dtype: float64
ikibz =    4, iqibz =    4
ibn
1    -91.272445
2   -212.636765
3    -26.879784
4    -26.879688
5    163.955047
6    163.954995
7   -107.550164
8   -191.966405
Name: en, dtype: float64


In [19]:
vbm_fm = fm[np.logical_and(fm["ikr"]==1,fm["ibn"]==4)]["en"].sum()/nq
vbm_dw = dw[np.logical_and(dw["ikr"]==1,dw["ibn"]==4)]["en"].sum()/nq
cbm_fm = fm[np.logical_and(fm["ikr"]==1,fm["ibn"]==5)]["en"].sum()/nq
cbm_dw = dw[np.logical_and(dw["ikr"]==1,dw["ibn"]==5)]["en"].sum()/nq

print("%10s%10s%10s%10s"%("","FM","DW","FM+DW"))
print("%10s%10.0f%10.0f%10.0f"%("VBM",vbm_fm,vbm_dw,vbm_fm+vbm_dw))
print("%10s%10.0f%10.0f%10.0f"%("CBM",cbm_fm,cbm_dw,cbm_fm+cbm_dw))
print("Direct gap ZPR = %10.0f"%((cbm_fm+cbm_dw)-(vbm_fm+vbm_dw)))

                  FM        DW     FM+DW
       VBM      -987      1114       127
       CBM     -1162      1000      -162
Direct gap ZPR =       -289


# EPCE computed with WELPH with my relaxed lattice constant

In [20]:
nqibz = 16
nkibz = 16
nk = 6**3
nq = 6**3
fm = pd.read_csv("./80Ry-myrelax/c.welph.save.pdep200/EPCE_DFT_FM.tab",
                   skiprows=1,names=["ikr","ibn","iq","imode","en"],sep="\\s+")
dw = pd.read_csv("./80Ry-myrelax/c.welph.save.pdep200/EPCE_DFT_DW.tab",
                   skiprows=1,names=["ikr","ibn","iq","imode","en"],sep="\\s+")
mappings = load_equivalence("./80Ry-myrelax/",nqibz)

In [21]:
for ikibz in [1,4]:
    for iqibz in [1,4]:
        filter1 = fm["ikr"] == ikibz
        filter2 = fm["iq"].isin(mappings[iqibz-1])
        fmpart = fm[filter1 & filter2]
        filter3 = dw["ikr"] == ikibz
        filter4 = dw["iq"].isin(mappings[iqibz-1])
        dwpart = dw[filter3 & filter4]
        q_degeneracy = len(mappings[iqibz-1])
        print("ikibz = %4d, iqibz = %4d"%(ikibz,iqibz))
#         print(fmpart[fmpart["ibn"]==1])
#         print(fmpart.groupby("ibn")["en"].sum()/q_degeneracy)
#         print(dwpart.groupby("ibn")["en"].sum()/q_degeneracy)    
        print(fmpart.groupby("ibn")["en"].sum()/q_degeneracy+dwpart.groupby("ibn")["en"].sum()/q_degeneracy)

ikibz =    1, iqibz =    1
ibn
1   -12.708269
2    25.194872
3    25.199256
4    25.198543
5   -14.899032
6   -14.908917
7   -14.907659
8   -31.340655
Name: en, dtype: float64
ikibz =    1, iqibz =    4
ibn
1    -54.463151
2    186.332934
3    186.345684
4    186.342620
5   -273.040716
6   -273.076696
7   -273.075611
8   -308.931955
Name: en, dtype: float64
ikibz =    4, iqibz =    1
ibn
1    -21.149150
2    -16.694735
3     10.147818
4     10.147769
5   -163.407883
6   -163.407697
7    325.661416
8    -26.222320
Name: en, dtype: float64
ikibz =    4, iqibz =    4
ibn
1    -91.077447
2   -212.890392
3    -26.894574
4    -26.894664
5    163.713653
6    163.718142
7   -107.434949
8   -190.797668
Name: en, dtype: float64
